In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium
import requests
from pandas.io.json import json_normalize

# Interface with GIS

In [ ]:
url = 'https://gis.atlantaga.gov/dpcd/rest/services/OpenDataService/FeatureServer/4/query?where=1%3D1&outFields=LOCALID,NAME,NPU,GLOBALID&outSR=4326&f=json'

In [ ]:
results = requests.get(url).json()
dataframe = json_normalize(results)

In [ ]:
data = dataframe['features'][0]
cleandata = json_normalize(data)
cleandata

In [ ]:
venues_map = folium.Map(tiles='Stamen Toner',location=[33.755845, -84.38902], zoom_start=10)

for index, row in cleandata.iterrows():
    
    name = row['attributes.NAME']
    geo_list = row['geometry.rings']
    coord = geo_list[0][:]
    for ll in coord:
        lat = ll[1]
        long = ll[0]
        folium.Circle(
            [lat, long],
            radius=3,
            fill=True
            ).add_to(venues_map)
#     print(ll)
    
# display map
venues_map

In [ ]:
import geopandas
npu = geopandas.GeoDataFrame.from_file('NPU.shp')

In [ ]:
import pyepsg

pyepsg.get(npu.crs['init'].split(':')[1])

# Interface with Foursquare
https://developer.foursquare.com/docs/api/venues/trending

In [2]:
# @hidden_cell
client_secret = 'PLV5C4EDNBSRH4AMBKOXNGHFZWHEWCGULGTQLPQZVQFJFYZO'
client_id = 'HE3V0C15YMFATHFBWRAZ0B32H12HYYSJ1HJ523U02CKPEO3M'
ver = '20180604'

In [ ]:
lat = 33.755845
long = -84.38901
limit = 50
radius=100000

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}&radius={}'.format(client_id, client_secret, lat, long, ver, limit,radius)

In [ ]:
results = requests.get(url).json()
print(results)
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items)

In [ ]:
dataframe.head()

In [ ]:
dataframe.shape

In [ ]:
dataframe.columns.values

In [ ]:
data=pd.DataFrame()
data['name'] = dataframe['venue.name']
data['categories'] = dataframe['venue.categories']
data['latitude'] = dataframe['venue.location.lat']
data['longitude'] = dataframe['venue.location.lng']
data['neighborhood'] = dataframe['venue.location.neighborhood']
data.neighborhood.unique()


In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
data['categories'] = data.apply(get_category_type, axis=1)

In [ ]:
data.head()

In [ ]:
data.dropna(subset=['latitude', 'longitude'])

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = data['neighborhood'].to_list()
neighborhood_labels = le.fit_transform(y)
data['neighborhood'] = neighborhood_labels
data.head()

In [ ]:
le = preprocessing.LabelEncoder()
y = data['categories'].to_list()
category_labels = le.fit_transform(y)
data['categories'] = category_labels
data.head()

In [ ]:
data.to_csv('training_data.csv')

### Visualize results

In [ ]:
cat_list = data.categories.unique()
colors = plt.cm.Dark2.colors
print(colors)
    

In [ ]:
venues_map = folium.Map(location=[lat, long], zoom_start=15)

for index, row in data.iterrows():
    lat = row['latitude']
    long = row['longitude']
    label = row['name']
    category = row['categories']
    
    folium.Circle(
        [lat, long],
        popup=label,
        radius=10,
        fill=True
        ).add_to(venues_map)

# display map
venues_map

# K Nearest Neighbors Classifer

In [ ]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
X = data['latitude'], data['longitude']
y = data['neighborhood']
clf = NearestCentroid()
clf.fit(X, y)
print(clf)

# Atlanta lat-long grid generation

In [3]:
min_lat = 33.619816
max_lat = 33.919599
min_long = -84.495203
max_long = -84.241616

In [4]:
def make_range(minval, maxval, steps):
    step = (maxval - minval)/steps
    val = minval
    outlist = list()
    outlist.append(minval)
    while val <= maxval:
        val += step
        outlist.append(val)
    return outlist

In [5]:
lat_list = make_range(min_lat, max_lat, 10)
long_list = make_range(min_long, max_long, 10)

In [11]:
from sklearn.model_selection import ParameterGrid
grid = {'latitude':lat_list, 'longitude':long_list}
grid = ParameterGrid(grid)
fsq_df = pd.DataFrame()
radius = 5000
limit=50
for set in grid:
    lat = set['latitude']
    long = set['longitude']
    print(lat, long)
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}&radius={}'.format(client_id, client_secret, lat, long, ver, limit,radius)
    results = requests.get(url).json()
    items = results['response']['groups'][0]['items']
    dataframe = json_normalize(items)
#     print(dataframe.head(1))
    data = pd.DataFrame()
    data['name'] = dataframe['venue.name']
    data['categories'] = dataframe['venue.categories']
    data['latitude'] = dataframe['venue.location.lat']
    data['longitude'] = dataframe['venue.location.lng']
    if 'venue.location.neighborhood' in list(dataframe.columns.values):
        data['neighborhood'] = dataframe['venue.location.neighborhood']
    else:
        data['neighborhood'] = np.nan
    print(data.head(2))    
    fsq_df.append(data,ignore_index=True)
    print(fsq_df.shape)
    
print(fsq_df.shape)
print(fsq_df.head())

33.619816 -84.495203
               name                                         categories  \
0         CFA  Cafe  [{'id': '4bf58dd8d48988d16d941735', 'name': 'C...   
1  Tropical Cuisine  [{'id': '4bf58dd8d48988d144941735', 'name': 'C...   

    latitude  longitude neighborhood  
0  33.613347 -84.489282          NaN  
1  33.622083 -84.476598          NaN  
(0, 0)
33.619816 -84.4698443
               name                                         categories  \
0        LA Fitness  [{'id': '4bf58dd8d48988d175941735', 'name': 'G...   
1  Tropical Cuisine  [{'id': '4bf58dd8d48988d144941735', 'name': 'C...   

    latitude  longitude neighborhood  
0  33.610833 -84.461797          NaN  
1  33.622083 -84.476598          NaN  
(0, 0)
33.619816 -84.44448560000001
         name                                         categories   latitude  \
0    QuikTrip  [{'id': '4d954b0ea243a5684a65b473', 'name': 'C...  33.608718   
1  LA Fitness  [{'id': '4bf58dd8d48988d175941735', 'name': 'G...  33.610833 

KeyboardInterrupt: 

In [ ]:
dataframe

In [7]:
list(dataframe.columns.values)

['reasons.count',
 'reasons.items',
 'referralId',
 'venue.categories',
 'venue.delivery.id',
 'venue.delivery.provider.icon.name',
 'venue.delivery.provider.icon.prefix',
 'venue.delivery.provider.icon.sizes',
 'venue.delivery.provider.name',
 'venue.delivery.url',
 'venue.events.count',
 'venue.events.summary',
 'venue.id',
 'venue.location.address',
 'venue.location.cc',
 'venue.location.city',
 'venue.location.country',
 'venue.location.crossStreet',
 'venue.location.distance',
 'venue.location.formattedAddress',
 'venue.location.labeledLatLngs',
 'venue.location.lat',
 'venue.location.lng',
 'venue.location.neighborhood',
 'venue.location.postalCode',
 'venue.location.state',
 'venue.name',
 'venue.photos.count',
 'venue.photos.groups',
 'venue.venuePage.id']